In [1]:
from download_delgado.delgado_datasets import DownloadAndConvertDelgadoDatasets
from mleap.data import Data
from mleap.data.estimators import instantiate_default_estimators
from mleap.experiments import TestOrchestrator
from mleap.analyze_results import AnalyseResults

### Step 1: Load the datasets

In [2]:
#load datasets as pandas DataFrame
#each dataset needs to have metadata attached to it containing the following keys:
# class_name: (string) name of column containing the column name in which the labels are stored
# dataset_name: (string) name of the dataset
delgado = DownloadAndConvertDelgadoDatasets()
datasets, metadata = delgado.download_and_extract_datasets(verbose = False) #think about storing in hdd

Error: Dataset Delgado_data\molec-biol-protein-second has a different number of arff files


### Step 2: Save the datasets in HDF5 database

In [3]:
# files io object needs to be used in test orch
# returned object needs to be integrated in code
data = Data()
data.pandas_to_db(save_loc_hdf5='delgado_datasets/', datasets=datasets, 
                  dts_metadata=metadata, save_loc_hdd='data/delgado.hdf5') #return files io object
input_io = data.open_hdf5('data/delgado.hdf5', mode='r')
out_io = data.open_hdf5('data/experiments.hdf5', mode='a')

### Step 3: Instantiate models and Test Orchestrator

In [4]:
instantiated_models = instantiate_default_estimators(estimators=['all'])

test_o = TestOrchestrator(hdf5_input_io=input_io, hdf5_output_io=out_io)

### Step 4: Split datasets

In [5]:
dts_names_list, dts_names_list_full_path = data.list_datasets(hdf5_io=input_io, hdf5_group='delgado_datasets/')
split_dts_list = data.split_datasets(hdf5_in=input_io, hdf5_out=out_io, dataset_paths=dts_names_list_full_path)

### Step 5: Run the experiments

In [6]:
test_o.run(input_io_datasets_loc=dts_names_list_full_path[0:3], output_io_split_idx_loc=split_dts_list[0:3], modelling_strategies=instantiated_models)

*** Training models on dataset: abalone. Total datasets processed: 0/3 ***
*** Training models on dataset: acute_inflammation. Total datasets processed: 1/3 ***
*** Training models on dataset: acute_nephritis. Total datasets processed: 2/3 ***


### Step 6: Analyze the results

In [7]:
analyze = AnalyseResults(hdf5_output_io=out_io, hdf5_input_io=input_io)
errors = analyze.calculate_loss_all_datasets(input_h5_original_datasets_group='delgado_datasets/', 
                                    output_h5_predictions_group='experiments/predictions/', 
                                    metric='mean_squared_error')

t_test, t_test_df = analyze.perform_t_test(errors)
print('******t-test******')
print(t_test_df)

# sign_test, sign_test_df = analyze.perform_sign_test()
# print('******sign test******')
# print(sign_test_df)

# t_test_bonferroni, t_test_bonferroni_df = analyze.perform_t_test_with_bonferroni_correction()
# print('******t-test bonferroni correction******')
# print(t_test_bonferroni_df)
# wilcoxon_test, wilcoxon_test_df = analyze.perform_wilcoxon()
# print('******Wilcoxon test******')
# print(wilcoxon_test_df)

# friedman_test, friedman_test_df = analyze.perform_friedman_test()
# print('******Friedman test******')
# print(friedman_test_df)

******t-test******
                                           pair  t_statistic  p_value
0      BernoulliNaiveBayes - GaussianNaiveBayes         2.92     0.57
1      BernoulliNaiveBayes - LogisticRegression         2.92     0.24
2  BernoulliNaiveBayes - RandomForestClassifier         2.92     0.14
3                     BernoulliNaiveBayes - SVC         2.92     0.16
4       GaussianNaiveBayes - LogisticRegression         2.92     0.07
5   GaussianNaiveBayes - RandomForestClassifier         2.92     0.09
6                      GaussianNaiveBayes - SVC         2.92     0.09
7   LogisticRegression - RandomForestClassifier         2.92     0.20
8                      LogisticRegression - SVC         2.92     0.14
9                  RandomForestClassifier - SVC         2.92     0.35
